In [ ]:
import logging
import os
import pathlib
from typing import Optional

import dspy
import pandas as pd
from dotenv import load_dotenv
from dspy.datasets import Dataset
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFewShot
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import pandas as pd

In [1]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

NameError: name 'pd' is not defined

In [110]:
llm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8090, url="http://127.0.0.1")
dspy.settings.configure(lm=llm)

In [104]:
path_env = pathlib.Path(os.getcwd()).parent.parent / '.env'
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

llm = dspy.OpenAI(
    model="gpt-4o", #"gpt-3.5-turbo" , #"gpt-4-0125-preview",  #gpt-4o-2024-05-13, #"gpt-4-1106-preview", # TODO: try turbo-instruct,
    max_tokens=1000)

# Assumes the Weaviate collection has a text key `content`
dspy.settings.configure(lm=llm, trace=[])

In [150]:
import pathlib
import pandas as pd
import numpy as np
from scipy import sparse
from sentence_transformers import SentenceTransformer
import faiss


def create_faiss_index(df, text_column, id_column, model_name="all-mpnet-base-v2", index_file="faiss_index.index"):
    """
    Create a FAISS index from a DataFrame containing text data.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    text_column (str): The name of the column containing text data.
    id_column (str): The name of the column containing unique identifiers for the texts.
    model_name (str): The name of the SentenceTransformer model to use for embeddings.
    index_file (str): The file path to save the FAISS index.

    Returns:
    index: The FAISS index object.
    model: The SentenceTransformer model used for embeddings.
    ids: List of document IDs.
    texts: List of document texts.
    """
    texts = df[text_column].tolist()
    ids = df[id_column].tolist()

    model = SentenceTransformer(model_name, device="cuda")

    # Calculate embeddings for the texts
    embeddings = model.encode(texts, show_progress_bar=False)

    # Create a FAISS index
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)  

    # Normalize embeddings to unit length and add to index
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    # Save the index to a file
    faiss.write_index(index, index_file)

    return index, model, ids, texts

In [5]:
class ContradictionsDataset(Dataset):

    def __init__(
        self,
        data_fpath: str,
        dev_size: Optional[float] = 0.2,
        test_size: Optional[float] = 0.2,
        input_keys: str = ["answer1", "answer2", "question"],
        seed: Optional[int] = 11235,
        *args,
        **kwargs
    ) -> None:
        """
        fact -> question
        """

        super().__init__(*args, **kwargs)

        self._train = []
        self._dev = []
        self._test = []

        # Read the training data
        train_data = pd.read_excel(pathlib.Path(data_fpath))[
            ["answer1", "answer2", "question", "faith_strict", "faithfulness"]]

        for col in train_data.columns:
            train_data[col] = train_data[col].apply(
                lambda x: str(x).strip("\n\t"))

        train_data["faith_strict"] = train_data["faith_strict"].apply(
            lambda x: int(x))
        train_data["faith_strict"] = train_data["faith_strict"].astype(str)

        train_data["faithfulness"] = train_data["faithfulness"].apply(
            lambda x: int(x))
        train_data["faithfulness"] = train_data["faithfulness"].astype(str)

        train_data = train_data[
            (train_data.faithfulness == "0") |
            (train_data.faithfulness == "1")
        
        ]

        train_data, temp_data = train_test_split(
            train_data, test_size=dev_size + test_size, random_state=seed)
        dev_data, test_data = train_test_split(
            temp_data, test_size=test_size / (dev_size + test_size), random_state=seed)

        self._train = [
            dspy.Example({**row}).with_inputs(*input_keys) for row in self._convert_to_json(train_data)
        ]
        self._dev = [
            dspy.Example({**row}).with_inputs(*input_keys) for row in self._convert_to_json(dev_data)
        ]
        self._test = [
            dspy.Example({**row}).with_inputs(*input_keys) for row in self._convert_to_json(test_data)
        ]

    def _convert_to_json(self, data: pd.DataFrame):
        if data is not None:
            return data.to_dict(orient='records')

In [6]:
class CheckAnswersFaithfulness(dspy.Signature):
    """Verify whether ANSWER1 and ANSWER2 are FAITHFUL (1) to each other or not (0) given QUESTION. If its faithfulness can't be determined, return 2."""

    QUESTION = dspy.InputField()
    ANSWER1 = dspy.InputField()
    ANSWER2 = dspy.InputField()
    faithfulness = dspy.OutputField(
        desc="predicted label (1,0, or 2 only)", prefix="Faithfulness:")
    rationale = dspy.OutputField(desc="explains the relation between ANSWER1 and ANSWER2", prefix="Rationale:")

class ClassifyContradiction(dspy.Signature):
    ("""Classify the contradiction between ANSWER1 and ANSWER2 given QUESTION into: """
    """(0) Discrepancy: The answers might be correct within their respective contexts, byt they offer conflicting guidance or explanations that could lead to confusion. """
    """(1) Strict: The two answers provide directly opposing information."""
    )
    
    QUESTION = dspy.InputField()
    ANSWER1 = dspy.InputField()
    ANSWER2 = dspy.InputField()
    contradiction_type = dspy.OutputField(
        desc="predicted label (0 or 1 only)", prefix="Contradiction_type:")
    rationale = dspy.OutputField(desc="explains the type of contradiction", prefix="Rationale:")

class QACheckerModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.checker = dspy.ChainOfThought(CheckAnswersFaithfulness)

    def process_faithfulness(self, faithfulness):

        try:
            if "0" in faithfulness:
                return 0
            elif "1" in faithfulness:
                return 1
            elif "2" in faithfulness:
                return 2
        except Exception as e:
            print(f"Error: {e}")
            print(f"Faithfulness: {faithfulness}")
            return faithfulness

    def forward(self, answer1, answer2, question):
        response = self.checker(ANSWER1=answer1, ANSWER2=answer2, QUESTION=question)
        print(f"-- -- faithfulness: {response.faithfulness}")
        print(f"-- -- rationale: {response.rationale}")

        return dspy.Prediction(faithfulness=self.process_faithfulness(response.faithfulness), rationale=response.rationale)

class ClassifyContradictionModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.contrad_classifier = dspy.ChainOfThought(ClassifyContradiction)

    def process_contrad_type(self, contrad_type):
        try:
            if "0" in contrad_type:
                return 0
            elif "1" in contrad_type:
                return 1
        except Exception as e:
            print(f"Error: {e}")
            print(f"Contrd_type: {contrad_type}")
            return contrad_type

    def forward(self, answer1, answer2, question):
        response = self.contrad_classifier(ANSWER1=answer1, ANSWER2=answer2, QUESTION=question)
        print(f"-- -- faithfulness: {response.contradiction_type}")
        print(f"-- -- rationale: {response.rationale}")

        return dspy.Prediction(contrad_type=self.process_contrad_type(response.contradiction_type), rationale=response.rationale)

In [7]:
mbd=4
mld=16
ncp=2
mr=5
dev_size=0.25
data_path = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/src/qa_system/tr_data/contradictions_dtset.xlsx"

def combined_score(example, pred, trace=None):

    pred_faith = pred["faithfulness"]
    ground_faith = int(example["faith_strict"]) if isinstance(example["faith_strict"], str) or isinstance(
        example["faith_strict"], float) else example["faith_strict"]

    return 1 if pred_faith == ground_faith else 0

def combined_score_class(example, pred, trace=None):

    pred_faith = pred["contrad_type"]
    ground_faith = int(example["faithfulness"]) if isinstance(example["faithfulness"], str) or isinstance(
        example["faithfulness"], float) else example["faithfulness"]

    return 1 if pred_faith == ground_faith else 0

In [8]:
dataset = ContradictionsDataset(
    data_fpath=data_path, dev_size=dev_size)


trainset = dataset._train
devset = dataset._dev
testset = dataset._test


config = dict(max_bootstrapped_demos=mbd, max_labeled_demos=mld,
              num_candidate_programs=ncp, max_rounds=mr)
teleprompter = BootstrapFewShotWithRandomSearch(
    metric=combined_score_class, **config)

compiled_pred = teleprompter.compile(
    ClassifyContradictionModule(), trainset=trainset, valset=devset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 2 candidate sets.


Average Metric: 3 / 3  (100.0):  18%|███████████████████████████████████████                                                                                                                                                                                | 2/11 [00:01<00:12,  1.36s/it]

-- -- faithfulness: Contradiction_type: 0
-- -- rationale: Both answers suggest introducing solid food around the same age (4 to 6 months), but they offer different types of food recommendations. ANSWER 1 suggests mixing breast milk with solid food, while ANSWER 2 suggests giving rice and cow's milk before introducing solid food. The discrepancy lies in the type of food recommended, which could lead to confusion for someone seeking guidance.
-- -- faithfulness: Contradiction_type: 1
-- -- rationale: The two answers provide directly opposing information. ANSWER 1 suggests that it is safe to use a sauna or hot tub during pregnancy for a short period, while ANSWER 2 states that using a sauna or hot tub during pregnancy should be avoided due to the risk of birth defects. These statements cannot both be true simultaneously, making this a strict contradiction.
-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The answers might be correct within their respective contexts. ANSWER 1 pr

Average Metric: 5 / 6  (83.3):  55%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                  | 6/11 [00:01<00:01,  4.58it/s]

-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The answers might be correct within their respective contexts. ANSWER 1 provides a specific weight limit (10 pounds) that pregnant women should avoid lifting, while ANSWER 2 suggests that lifting moderate weights can be safe depending on the woman's fitness level. These answers offer conflicting guidance that could lead to confusion, but they are not directly opposing; rather, they reflect different perspectives or guidelines on the same issue.
-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The answers provide conflicting guidance on the amount of food intake during pregnancy. ANSWER 1 suggests that one should "eat for two," implying a significant increase in food intake, while ANSWER 2 emphasizes the importance of a balanced diet without necessarily increasing the quantity of food. Both answers might be correct within their respective contexts, but they offer different approaches that could lead to confusion.
-- --

...

-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The two answers might be correct within their respective contexts but offer conflicting guidance that could lead to confusion. ANSWER 1 provides a specific recommendation of drinking at least 8 glasses of water daily, while ANSWER 2 suggests that fluid needs can vary and women should drink according to their thirst. These statements do not directly oppose each other but offer different approaches to hydration during pregnancy, leading to a discrepancy.
-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The two answers provide conflicting guidance on the introduction of solid food to an infant, but they might be correct within their respective contexts. ANSWER 1 suggests introducing solid food mixed with breast milk at 4 to 6 months of age, while ANSWER 2 mentions giving rice and cow's milk to six-month-old babies before introducing solid food. The discrepancy lies in the specific types of food and the timing, which coul

Average Metric: 7 / 8  (87.5):  73%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                           | 8/11 [00:02<00:00,  3.95it/s]

-- -- faithfulness: Contradiction_type: 1
-- -- rationale: The two answers provide directly opposing information. ANSWER 1 states that breastfeeding can reduce the risk of breast cancer for the mother, while ANSWER 2 claims that breastfeeding has no significant impact on the mother's risk of developing breast cancer. These statements cannot both be true simultaneously, making this a strict contradiction.
-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The two answers might be correct within their respective contexts but offer conflicting guidance that could lead to confusion. ANSWER 1 suggests that raw fish is a typical food eaten during pregnancy, while ANSWER 2 lists lentils, ghee, nuts, and milk as the foods pregnant women eat, implying a focus on healthy foods. These answers do not directly oppose each other but provide different dietary recommendations, leading to a discrepancy.


Average Metric: 8 / 11  (72.7): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  4.12it/s]

-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The answers might be correct within their respective contexts but offer conflicting guidance that could lead to confusion. ANSWER 1 suggests that pregnant women should eat liver for its high vitamin A content, implying it is beneficial. ANSWER 2, on the other hand, warns that high vitamin A intake from liver can be harmful during pregnancy. These statements do not directly oppose each other but provide conflicting advice, making this a discrepancy.
-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 states that women must not cry out or scream during labour, which suggests a strict rule or guideline. ANSWER 2, on the other hand, acknowledges that women's behavior during labor varies based on personality, indicating that some may indeed scream or make noise. This could lead to confusion as one answer implies a strict

In [9]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=50)

In [10]:
evaluator(compiled_pred, metric=combined_score_class)

Average Metric: 1 / 1  (100.0):  11%|████████████████████████                                                                                                                                                                                                | 1/9 [00:01<00:09,  1.15s/it]

-- -- faithfulness: Contradiction_type: 1
-- -- rationale: The two answers provide directly opposing information. ANSWER 1 states that pregnant women should avoid all herbal teas, while ANSWER 2 asserts that some herbal teas are safe and even beneficial during pregnancy. These statements cannot both be true simultaneously, making this a strict contradiction.


Average Metric: 2 / 2  (100.0):  22%|████████████████████████████████████████████████                                                                                                                                                                        | 2/9 [00:02<00:08,  1.22s/it]

-- -- faithfulness: Contradiction_type: 1
-- -- rationale: The two answers provide directly opposing information. ANSWER 1 states that it is important to avoid nuts during pregnancy due to the risk of allergies, while ANSWER 2 asserts that there is no evidence that eating nuts during pregnancy increases the risk of allergies in the baby. These statements cannot both be true simultaneously, making this a strict contradiction.


Average Metric: 2 / 3  (66.7):  33%|████████████████████████████████████████████████████████████████████████▎                                                                                                                                                | 3/9 [00:04<00:08,  1.39s/it]

...

-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 suggests that consuming fish oil can benefit the baby's brain development, while ANSWER 2 warns against fish oil supplements due to potential risks. These statements do not directly oppose each other but provide conflicting advice that could lead to confusion. Therefore, this is a discrepancy.


Average Metric: 5 / 9  (55.6): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:12<00:00,  1.44s/it]

-- -- faithfulness: Contradiction_type: 0
-- -- rationale: The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 states that cosmetic dental procedures are safe during pregnancy, while ANSWER 2 advises postponing non-essential dental procedures until after pregnancy. This could lead to confusion, but it is not a strict contradiction as both statements could be true depending on the specific circumstances or differing medical opinions.
Average Metric: 5 / 9  (55.6%)


,answer1,answer2,question,faith_strict,faithfulness,contrad_type,rationale,combined_score_class
0,Pregnant women should avoid all herbal teas.,Some herbal teas are safe and even beneficial during pregnancy.,Is it safe to drink herbal teas while pregnant?,0,1,1,"The two answers provide directly opposing information. ANSWER 1 states that pregnant women should avoid all herbal teas, while ANSWER 2 asserts that some herbal...",1
1,It's important to avoid nuts during pregnancy due to the risk of allergies.,There is no evidence that eating nuts during pregnancy increases the risk of allergies in the baby.,Should I avoid nuts during pregnancy?,0,1,1,"The two answers provide directly opposing information. ANSWER 1 states that it is important to avoid nuts during pregnancy due to the risk of allergies,...",1
2,It’s normal for pregnant women to experience shortness of breath.,Shortness of breath during pregnancy can indicate a serious problem and should be checked.,Is shortness of breath normal during pregnancy?,0,1,0,The two answers might be correct within their respective contexts but offer conflicting guidance that could lead to confusion. ANSWER 1 states that shortness of...,0
3,"It's common to experience mild swelling during pregnancy, especially in the ankles and feet.",Swelling in the ankles and feet is abnormal during pregnancy and should be checked by a healthcare provider.,Is swelling in the ankles and feet during pregnancy normal?,0,1,1,"The two answers provide directly opposing information. ANSWER 1 states that it is common to experience mild swelling in the ankles and feet during pregnancy,...",1
4,Sleeping on your back during pregnancy is unsafe as it can restrict blood flow.,"Sleeping on your back is generally safe, but side sleeping is preferred.",Is it safe to sleep on my back while pregnant?,0,1,0,The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 states that sleeping on your back during pregnancy is...,0
5,"The shot, which temporarily decreases bone density while in use, is probably not the best option for people with disabilities that affect bone density.",People with disabilities that affect bone density may find the shot beneficial.,Does the shot decrease bone density for people with disabilities?,0,1,0,The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 focuses on the potential negative impact of the shot...,0
6,All pregnant women need to take iron supplements.,Iron supplements are only necessary if the woman is diagnosed with iron deficiency anemia.,Should I take iron supplements during pregnancy?,0,0,0,The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 suggests that all pregnant women need to take iron...,1
7,Consuming a diet high in fish oil during pregnancy can help boost the baby's brain development.,Fish oil supplements are unnecessary during pregnancy and could increase the risk of bleeding.,Is it beneficial for pregnant women to consume fish oil?,0,0,0,The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 suggests that consuming fish oil can benefit the baby's...,1
8,Cosmetic dental procedures are safe during pregnancy.,Non-essential dental procedures should be postponed until after pregnancy.,Can I undergo cosmetic dental procedures while pregnant?,0,1,0,"The two answers might be correct within their respective contexts but offer conflicting guidance. ANSWER 1 states that cosmetic dental procedures are safe during pregnancy,...",0


55.56

In [13]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def segment_facts(sentence):
    # Process the sentence with spaCy
    doc = nlp(sentence)
    
    # Identify the subject
    subject = None
    for token in doc:
        if token.dep_ == "nsubj":
            subject = token.text
            break

    if not subject:
        # Fallback: if no clear subject, try the first proper noun or noun
        for token in doc:
            if token.pos_ in ("PROPN", "NOUN"):
                subject = token.text
                break

    if not subject:
        # If no subject found, return the original sentence as the only fact
        return [sentence]

    # Initialize a list to store the segmented facts
    segmented_facts = []

    # Find the root verb/action in the sentence
    for token in doc:
        if token.dep_ == "ROOT":
            action = token.text
            break

    # Construct sentences from conjuncts, objects, and other dependencies
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ in ("dobj", "pobj", "attr"):
            segmented_facts.append(f"{subject} {action} {chunk.text}.")

    # Handle compound actions and conjunctions
    for token in doc:
        if token.dep_ == "conj":
            segmented_facts.append(f"{subject} {action} {token.text}.")
        elif token.dep_ == "prep":
            obj = " ".join([child.text for child in token.children])
            segmented_facts.append(f"{subject} {action} {token.text} {obj}.")

    return segmented_facts

# Example sentences to segment
sentences = [
    "Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.",
    "Diabetes mellitus is a metabolic disease that causes high blood sugar.",
    "Hypertension is a condition that can lead to heart disease, stroke, and kidney failure."
]

# Process each sentence
for sentence in sentences:
    facts = segment_facts(sentence)
    for fact in facts:
        print(fact)
    print("-" * 40)


that Chorioamnionitis pregnancy.
that Chorioamnionitis unexplained fever.
that Chorioamnionitis uterine tenderness.
that Chorioamnionitis the fetus.
that Chorioamnionitis the woman.
that Chorioamnionitis during pregnancy.
that Chorioamnionitis with tenderness.
that Chorioamnionitis rate.
that Chorioamnionitis in fetus.
that Chorioamnionitis rate.
that Chorioamnionitis in woman.
that Chorioamnionitis discharge.
----------------------------------------
mellitus is a metabolic disease.
mellitus is high blood sugar.
----------------------------------------
Hypertension is a condition.
Hypertension is heart disease.
Hypertension is to disease.
Hypertension is stroke.
----------------------------------------


In [15]:
import spacy
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

nlp = spacy.load("en_core_web_sm")
model_name = "biodatlab/score-claim-identification"
tokenizer_name = "allenai/scibert_scivocab_uncased"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def inference(abstract: str):
    """
    Split an abstract into sentences and perform claim identification.
    """
    if abstract.strip() == "":
        return "Please provide an abstract as an input."
    claims = []
    sents = [sent.text for sent in nlp(abstract).sents]  # a list of sentences
    inputs = tokenizer(
        sents,
        return_tensors="pt",
        truncation=True,
        padding="longest"
    )
    logits = model(**inputs).logits
    preds = logits.argmax(dim=1)  # convert logits to predictions
    claims = [sent for sent, pred in zip(sents, preds) if pred == 1]
    if len(claims) > 0:
        return ".\n".join(claims)
    else:
        return "No claims found from a given abstract."

claims = inference(sentences[0])  # string of claim joining with \n


/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/home/lbartolome/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [4]:
!export CUDA_VISIBLE_DEVICES=2

In [22]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
hft = T5TokenizerFast.from_pretrained('varadhbhatnagar/fc-claim-det-T5-base')
hfm = T5ForConditionalGeneration.from_pretrained('varadhbhatnagar/fc-claim-det-T5-base').to("cpu")
row = 'Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.'
row2 = "Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia."
tokenized_text = hft.encode(row2, return_tensors="pt")
summary_ids = hfm.generate(tokenized_text,
                                  num_beams=10,
                                  no_repeat_ngram_size=2,
                                  min_length=5,
                                  max_length=100,
                                  early_stopping=True)

output = hft.decode(summary_ids[0], skip_special_tokens=True)


In [23]:
output

'agammaglobulinemia is caused by faulty X chromosomes.'

In [10]:
from transformers import pipeline

classifier = pipeline(
        "text-classification", 
        model="Nithiwat/bert-base_claimbuster"
)

classifier("there is no earthquake in turkey")        

config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9970910549163818}]

In [11]:
def initalize_model():
    model = pipeline('summarization', model='ainize/bart-base-cnn') #  google/pegasus-xsum
    return model

def summarise(text, model):
    summary = model(text, min_length=30, do_sample=False)

    return summary[0]

# initialize ML model
model = initalize_model()

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [16]:
summary = summarise("Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia.", model)
summary

Your max_length is set to 128, but your input_length is only 87. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


{'summary_text': 'Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child .\nThis is true for every pregnancy .\nIf a daughter gets the gene, she will likely be a healthy carrier like her mother .'}

In [15]:
pred = classifier(output)[0]
print(pred)



{'label': 'LABEL_0', 'score': 0.8251485824584961}


In [31]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
model = T5ForConditionalGeneration.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
summary = 'Simone Biles made a triumphant return to the Olympic stage at the Paris 2024 Games, competing in the women’s gymnastics qualifications. Overcoming a previous struggle with the “twisties” that led to her withdrawal from events at the Tokyo 2020 Olympics, Biles dazzled with strong performances on all apparatus, helping the U.S. team secure a commanding lead in the qualifications. Her routines showcased her resilience and skill, drawing enthusiastic support from a star-studded audience'

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

In [34]:
tok_input = tokenizer.batch_encode_plus([row2], return_tensors="pt", padding=True)
claims = model.generate(**tok_input)
claims = tokenizer.batch_decode(claims, skip_special_tokens=True)
claims

['Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia.']

In [37]:
pred = classifier(claims)[0]
print(pred)

{'label': 'LABEL_1', 'score': 0.9928246736526489}


In [41]:
import pathlib
import pandas as pd
import numpy as np
from scipy import sparse
print("Loading data...")
path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/multi_blade_filtered/df_1.parquet")

path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/MULTI_BLADE_FILTERED/ldatm_rosie_1_20")
path_corpus_en = path_model / "train_data" / "corpus_EN.txt"
path_corpus_es = path_model / "train_data" / "corpus_ES.txt"

persist_directory = (path_model / 'db_contr_mono').as_posix()

raw = pd.read_parquet(path_source)
with path_corpus_en.open("r", encoding="utf-8") as f:
    lines = [line for line in f.readlines()]
corpus_en = [line.rsplit(" 0 ")[1].strip().split() for line in lines]

ids = [line.split(" 0 ")[0] for line in lines]
df_en = pd.DataFrame({"lemmas": [" ".join(doc) for doc in corpus_en]})
df_en["doc_id"] = ids
df_en["len"] = df_en['lemmas'].apply(lambda x: len(x.split()))
df_en["id_top"] = range(len(df_en))
df_en_raw = df_en.merge(raw, how="inner", on="doc_id")[["doc_id", "id_top", "id_preproc", "lemmas_x", "text", "len"]]

# Read thetas 
thetas = sparse.load_npz(path_model.joinpath(f"mallet_output/{'EN'}/thetas.npz")).toarray()
betas = np.load((path_model.joinpath(f"mallet_output/{'EN'}/betas.npy")))
def get_thetas_str(row,thetas):
    return " ".join([f"{id_}|{round(el, 4)}" for id_,el in enumerate(thetas[row]) if el!=0.0])

def get_most_repr_tpc(row,thetas):
    return np.argmax(thetas[row])

# Save thetas in dataframe and "assigned topic"
df_en_raw["thetas"] = df_en_raw.apply(lambda row: get_thetas_str(row['id_top'], thetas), axis=1)
df_en_raw["id_tpc"] = df_en_raw.apply(lambda row: get_most_repr_tpc(row['id_top'], thetas), axis=1)
tpc = 0
df_tpc = df_en_raw[df_en_raw.id_tpc == tpc]

Loading data...


In [3]:
import pathlib
import pandas as pd
import numpy as np
from scipy import sparse
print("Loading data...")
path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/multi_blade_filtered/df_1.parquet")

path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/MULTI_BLADE_FILTERED/ldatm_rosie_1_20")
path_corpus_en = path_model / "train_data" / "corpus_EN.txt"
path_corpus_es = path_model / "train_data" / "corpus_ES.txt"

persist_directory = (path_model / 'db_contr_mono').as_posix()

raw = pd.read_parquet(path_source)
with path_corpus_es.open("r", encoding="utf-8") as f:
    lines = [line for line in f.readlines()]
corpus_es = [line.rsplit(" 0 ")[1].strip().split() for line in lines]

ids = [line.split(" 0 ")[0] for line in lines]
df_en = pd.DataFrame({"lemmas": [" ".join(doc) for doc in corpus_es]})
df_en["doc_id"] = ids
df_en["len"] = df_en['lemmas'].apply(lambda x: len(x.split()))
df_en["id_top"] = range(len(df_en))
df_en_raw = df_en.merge(raw, how="inner", on="doc_id")[["doc_id", "id_top", "id_preproc", "lemmas_x", "text", "len"]]

# Read thetas 
thetas = sparse.load_npz(path_model.joinpath(f"mallet_output/{'ES'}/thetas.npz")).toarray()
betas = np.load((path_model.joinpath(f"mallet_output/{'ES'}/betas.npy")))
def get_thetas_str(row,thetas):
    return " ".join([f"{id_}|{round(el, 4)}" for id_,el in enumerate(thetas[row]) if el!=0.0])

def get_most_repr_tpc(row,thetas):
    return np.argmax(thetas[row])

# Save thetas in dataframe and "assigned topic"
df_en_raw["thetas"] = df_en_raw.apply(lambda row: get_thetas_str(row['id_top'], thetas), axis=1)
df_en_raw["id_tpc"] = df_en_raw.apply(lambda row: get_most_repr_tpc(row['id_top'], thetas), axis=1)
tpc = 0
df_tpc_es = df_en_raw[df_en_raw.id_tpc == tpc]

Loading data...


In [45]:
df_tpc.head()

,doc_id,id_top,id_preproc,lemmas_x,text,len,thetas,id_tpc
20,EN_513461_85839-22,20,14328,help baby fall_asleep baby know sleep informat...,How can you help your baby fall asleep: Not al...,38,0|0.9968000054359436 12|0.0031999999191612005,0
46,EN_1393570_301605-29,46,34634,sure cover baby head increase baby risk baby p...,Make sure nothing is covering your baby's head...,103,0|1.0,0
82,EN_1386177_300946-35,82,63331,baby sleep firm_flat mattress_firm surface_sla...,"Your baby should sleep on a firm, flat mattres...",85,0|0.8909000158309937 1|0.012600000016391277 3|...,0
86,EN_977522_167004-14,86,67739,sleep pattern sleep habit influence baby tempe...,Sleep patterns: Sleep habits are influenced by...,17,0|0.5479000210762024 10|0.003700000001117587 1...,0
118,EN_172959_7099-5,118,97719,information technology work parent_grandparent...,Why it works: We found that many parents and g...,24,0|0.4487000107765198 1|0.18140000104904175 10|...,0


In [133]:
df_tpc_es

,doc_id,id_top,id_preproc,lemmas_x,text,len,thetas,id_tpc
9,ES_584418_68197-9,9,224,hijo talasemia_hbss dispuesto proporcionar muestra sangre orina crisis dolor analizar cambio sangre orina monitorear producir lesión renal_agudo crisis dolor síndrome torácico agudo participante año ingresar crisis dolor síndrome torácico agudo cabo_birmingham alabama,¿Usted o su hijo tienen talasemia HbSS o Sβ0 y están dispuestos a proporcionar muestras de sangre y orina durante una crisis de dolor? Este estudio analizará si los cambios en la sangre y la orina se pueden utilizar para monitorear si se ha producido lesión renal aguda como resultado de una crisis de dolor o síndrome torácico agudo. Los participantes de este estudio deben tener entre 1 y 25 años y deben ser ingresados con una crisis de dolor o síndrome torácico agudo. Este estudio se lleva a cabo en Birmingham (Alabama).,32,0|0.75 19|0.25,0
44,ES_603718_75256-10,44,841,hijo recuento_plaquetario investigar_recuento plaqueta_inmadura marcador riesgo sangrado nacido_trombocitopenia comparación_recuento plaquetario_plaqueta inmadura_plaqueta producido_marcador producción_plaqueta participar nacido semana edad_gestacional peso_nacer gramo_recuento plaquetario cabo múltiple unidos_países bajos_suecia hagar_clic nombre_ensayo sitio_web ensayo_clínico nih_inglés,"¿Tiene su hijo un recuento plaquetario bajo? Este estudio investiga los recuentos de plaquetas inmaduras como un marcador del riesgo de sangrado en recién nacidos con trombocitopenia, en comparación con los recuentos plaquetarios solos. Las plaquetas inmaduras son las plaquetas producidas más recientemente y posiblemente sean un mejor marcador de la producción de plaquetas. Para participar en este estudio, debe tener un recién nacido de menos de 32 semanas de edad gestacional, debe tener un peso al nacer de más de 500 gramos, y tener un recuento plaquetario de menos de 100 × 109/l. Este estudio se lleva a cabo en múltiples lugares de los Estados Unidos, los Países Bajos y Suecia. Haga clic en el nombre del ensayo para obtener más información en el sitio web de ensayos clínicos de los NIH (en inglés).",29,0|1.0,0
70,ES_592711_74128-1,70,1254,analizar medicamento llamado_metformina ayudar tratar aneurisma_aórtico abdomen diagnóstico amágenes_metformina ayudar prevenir aumento tamaño aneurisma_aórtico participar año diagnosticar aneurisma_aorta abdominal cabo stanford_california hagar_clic nombre_ensayo sitio_web ensayo_clínico nih_inglés,"Este estudio analiza si un medicamento llamado metformina puede ayudar a tratar los aneurismas aórticos en el abdomen. El estudio utilizará estudios de diagnóstico por imágenes para examinar si la metformina puede ayudar a prevenir el aumento de tamaño de estos aneurismas aórticos. Para participar en este estudio, debe tener entre 55 y 90 años y habérsele diagnosticado un aneurisma de aorta abdominal. Este estudio se lleva a cabo en Stanford (California). Haga clic en el nombre del ensayo para obtener más información en el sitio web de ensayos clínicos de los NIH (en inglés).",26,0|1.0,0
156,ES_486251_63877-48,156,2817,paciente participar ensayo_clínico ayudar mejorar forma tratar cáncer futuro ensayo_clínico tratamiento eficaz responder_pregunta importante ayudar avanzar,"Los pacientes que participan en los ensayos clínicos también ayudan a mejorar la forma en que se tratará el cáncer en el futuro. Aunque los ensayos clínicos no siempre llevan a tratamientos eficaces, a menudo responden a preguntas importantes y ayudan a avanzar en la investigación.",16,0|0.8104000091552734 9|0.06369999796152115 17|0.125900000333786,0
252,ES_593115_74164-9,252,4712,hijo anemia célula_falciform interés contribuir_recopilación dato plazo monitorear paciente recoger_muestra biológico paciente enfermedad célula_falciform comprender evolución plazo participante edad enfermedad célula_falciform cabo centro illinois_luisiana carolina_norte tennessee,"¿Usted o su hijo tienen la anemia de células falciformes y también, in

In [152]:
index_en, model_en, ids_en, texts_en = create_faiss_index(df_tpc, text_column='text', id_column='doc_id', index_file='faiss_index_en.index')
index_es, model_es, ids_es, texts_es = create_faiss_index(df_tpc_es, text_column='text', id_column='doc_id', index_file='faiss_index_es.index')

In [71]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
import itertools

tokenizer = T5Tokenizer.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
claim_extractor = T5ForConditionalGeneration.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
classifier = pipeline(
        "text-classification", 
        model="Nithiwat/bert-base_claimbuster"
)

#lens = [len(c) for c in claims]
#texts = [[text[i]]*lens[i] for i in range(len(lens))]
#scores = [classifier(c)[0]['label'] for c in claims] 

def generate_claims(text):
    tok_input = tokenizer.batch_encode_plus([text], return_tensors="pt", padding=True)
    claims = claim_extractor.generate(**tok_input)
    claims = tokenizer.batch_decode(claims, skip_special_tokens=True)[0].split(".")

    scores = [classifier(c)[0]['label'] for c in claims] 

    return claims, scores

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [52]:
df_tpc.text.values.tolist()[0]

'How can you help your baby fall asleep: Not all babies know how to put themselves to sleep. When it is time for bed, many parents want to rock or breastfeed a baby to sleep. Establishing a routine at bedtime is a good idea. However, be sure that the baby does not fall asleep while eating or in your arms. This may become a pattern and the baby may begin to expect to be in your arms in order to fall asleep. When the baby briefly awakens during a sleep cycle, he or she may not be able to go back to sleep on his or her own.'

In [81]:
all_ids = []
all_texts = []
all_claims = []
all_scores = []

for id_, el in df_tpc.iterrows():
    claims, scores = generate_claims(el.text)
    texts = [el.text]*len(claims)
    ids = [el.doc_id]*len(claims)

    all_ids.append(ids)
    all_texts.append(texts)
    all_claims.append(claims)
    all_scores.append(scores)


results = pd.DataFrame(
    {
        "ids": list(itertools.chain(*all_ids)),
        "text" : list(itertools.chain(*all_texts)),
        "claims" : list(itertools.chain(*all_claims)),
        "score" : list(itertools.chain(*all_scores))
    }
)


In [85]:
filtered = results[results.score == "LABEL_1"].drop_duplicates(subset=['claims'], keep='first', inplace=False)

filtered

,ids,text,claims,score
0,EN_513461_85839-22,"How can you help your baby fall asleep: Not all babies know how to put themselves to sleep. When it is time for bed, many parents want to rock or breastfeed a baby to sleep. Establishing a routine at bedtime is a good idea. However, be sure that the baby does not fall asleep while eating or in your arms. This may become a pattern and the baby may begin to expect to be in your arms in order to fall asleep. When the baby briefly awakens during a sleep cycle, he or she may not be able to go back to sleep on his or her own.",Not all babies know how to put themselves to sleep,LABEL_1
9,EN_1393570_301605-29,"Make sure nothing is covering your baby's head. These increase a baby's risk of suffocating\n- Put your baby in other positions while they are awake. This helps your baby grow stronger. It also helps prevent your baby from having a misshaped head. When your baby is awake, hold your baby. Give your baby time on their tummy while awake and supervised for short periods of time beginning soon after coming home from the hospital. Slowly increase tummy time to at least 15 to 30 minutes each day by 7 weeks old. Try not to let your baby sit in a seat or swing for long periods of time\n- Don't using sitting devices for routine sleep. Infant seats, car seats, strollers, infant carriers, and infant swings are not advised for routine sleep. These may lead to blockage of a baby's airway or suffocation. If your baby is in a sitting device, remove them from the device and put them in the crib or other appropriate surface as soon as is safe and practical\n- Make sure your baby doesn't get overheated when sleeping. Keep the room at a temperature that is comfortable for you and your baby. Dress your baby lightly. Instead of using blankets, keep your baby warm by dressing them in a sleep sack, or a wearable blanket. Don't use a hat on your baby indoors\n- Use caution when swaddling your baby. Swaddling doesn't reduce the risk for SIDS. If you choose to swaddle your baby, make sure they are on their back and the swaddle is not too tight.",These increase a baby's risk of suffocating,LABEL_1
31,EN_1386177_300946-35,"Your baby should sleep on a firm, flat mattress or firm surface with no slant. Cover the mattress with a fitted sheet. Don’t use fluffy blankets or comforters. Don’t let your baby sleep on a waterbed, air mattress, sofa, sheepskin, pillow, or other soft material. Don’t put soft toys, pillows, or bumper pads in the crib\n- Not overheating. Keep your baby warm but not too warm. The temperature in your baby’s room should feel comfortable to you. Don't overbundle, overdress, or cover a baby's face or head. Don't put a hat on your baby when indoors\n- Sharing a room. The American Academy of Pediatrics advises that babies sleep close to the parent's bed, but in a separate crib or bassinet for babies. This is advised ideally for the baby's first year. But you should do this at least for the first 6 months\n- Not sharing a bed. Don't put your baby to sleep in a bed with other children. Don’t put your baby to sleep on a sofa, either alone or with another person. Don't share your bed with your baby, especially if you are using alcohol or other drugs. You can bring your baby to your bed for feedings and comforting. But return your baby to the crib for sleep. Bed sharing is also not advised for twins or other multiples\n- Not allowing smoking around your baby. The risk of SIDS is higher for babies whose mothers smoked during pregnancy.",The risk of SIDS is higher for babies whose mothers whose mothers smoked during pregnancy,LABEL_1
43,EN_535414_88300-16,"Helping your baby fall asleep: Babies may not be able to establish their own sleeping and waking patterns. Surprisingly, not all babies know how to put themselves to sleep, or are able to go back to sleep if they are awakened in the night. When it is time for bed, many parents want to rock or breastfeed a baby to help them fall asleep. Establishing a rout

In [125]:
class GenerateQuestion(dspy.Signature):
    fact = dspy.InputField()
    context = dspy.InputField()
    question = dspy.OutputField(desc="it asks the fact", prefix="Question:")


class QAGeneratorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_question = dspy.Predict("fact,context->question")

    def forward(self, fact,context):
        question = self.generate_question(fact=fact, context=context).question
        return dspy.Prediction(question=question)

qa_gen = QAGeneratorModule()

In [158]:
class TranslatorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.translate = dspy.Predict("english->spanish")

    def forward(self, english):
        spanish = self.translate(english=english).spanish
        return spanish
tr = TranslatorModule()
tr("What is the best way to help your baby fall asleep?")

'¿Cómo puedo ayudar a mi bebé a dormir?'

In [126]:
questions = [qa_gen(q,con).question for q,con in zip(filtered.claims.values.tolist(),filtered.text.values.tolist())]
filtered["question"] = questions

In [154]:
filtered["len"] = filtered["question"].apply(lambda x: len(x))
filtered_2 = filtered[
    (filtered["len"] <= 300) &
    (filtered["question"].str.endswith("?"))
].drop_duplicates(subset=['question'], keep='first', inplace=False)
filtered_2.to_excel("test_tpc12.xlsx")

In [155]:
filtered_2

,ids,text,claims,score,question,len
0,EN_513461_85839-22,"How can you help your baby fall asleep: Not all babies know how to put themselves to sleep. When it is time for bed, many parents want to rock or breastfeed a baby to sleep. Establishing a routine at bedtime is a good idea. However, be sure that the baby does not fall asleep while eating or in your arms. This may become a pattern and the baby may begin to expect to be in your arms in order to fall asleep. When the baby briefly awakens during a sleep cycle, he or she may not be able to go back to sleep on his or her own.",Not all babies know how to put themselves to sleep,LABEL_1,What is the best way to help your baby fall asleep?,51
9,EN_1393570_301605-29,"Make sure nothing is covering your baby's head. These increase a baby's risk of suffocating\n- Put your baby in other positions while they are awake. This helps your baby grow stronger. It also helps prevent your baby from having a misshaped head. When your baby is awake, hold your baby. Give your baby time on their tummy while awake and supervised for short periods of time beginning soon after coming home from the hospital. Slowly increase tummy time to at least 15 to 30 minutes each day by 7 weeks old. Try not to let your baby sit in a seat or swing for long periods of time\n- Don't using sitting devices for routine sleep. Infant seats, car seats, strollers, infant carriers, and infant swings are not advised for routine sleep. These may lead to blockage of a baby's airway or suffocation. If your baby is in a sitting device, remove them from the device and put them in the crib or other appropriate surface as soon as is safe and practical\n- Make sure your baby doesn't get overheated when sleeping. Keep the room at a temperature that is comfortable for you and your baby. Dress your baby lightly. Instead of using blankets, keep your baby warm by dressing them in a sleep sack, or a wearable blanket. Don't use a hat on your baby indoors\n- Use caution when swaddling your baby. Swaddling doesn't reduce the risk for SIDS. If you choose to swaddle your baby, make sure they are on their back and the swaddle is not too tight.",These increase a baby's risk of suffocating,LABEL_1,What are some ways to reduce the risk of SIDS?,46
31,EN_1386177_300946-35,"Your baby should sleep on a firm, flat mattress or firm surface with no slant. Cover the mattress with a fitted sheet. Don’t use fluffy blankets or comforters. Don’t let your baby sleep on a waterbed, air mattress, sofa, sheepskin, pillow, or other soft material. Don’t put soft toys, pillows, or bumper pads in the crib\n- Not overheating. Keep your baby warm but not too warm. The temperature in your baby’s room should feel comfortable to you. Don't overbundle, overdress, or cover a baby's face or head. Don't put a hat on your baby when indoors\n- Sharing a room. The American Academy of Pediatrics advises that babies sleep close to the parent's bed, but in a separate crib or bassinet for babies. This is advised ideally for the baby's first year. But you should do this at least for the first 6 months\n- Not sharing a bed. Don't put your baby to sleep in a bed with other children. Don’t put your baby to sleep on a sofa, either alone or with another person. Don't share your bed with your baby, especially if you are using alcohol or other drugs. You can bring your baby to your bed for feedings and comforting. But return your baby to the crib for sleep. Bed sharing is also not advised for twins or other multiples\n- Not allowing smoking around your baby. The risk of SIDS is higher for babies whose mothers smoked during pregnancy.",The risk of SIDS is higher for babies whose mothers whose mothers smoked during pregnancy,LABEL_1,What are the risk factors for SIDS?,35
86,EN_992647_17713-5,"There is evidence that early introduction of pacifiers negatively impacts breastfeeding outcomes. However, pacifier use during the birth hospitalization can provide comfort during painful procedures (e

In [ ]:
###

In [4]:
df_tpc_es

,doc_id,id_top,id_preproc,lemmas_x,text,len,thetas,id_tpc
9,ES_584418_68197-9,9,224,hijo talasemia_hbss dispuesto proporcionar mue...,¿Usted o su hijo tienen talasemia HbSS o Sβ0 y...,32,0|0.75 19|0.25,0
44,ES_603718_75256-10,44,841,hijo recuento_plaquetario investigar_recuento ...,¿Tiene su hijo un recuento plaquetario bajo? E...,29,0|1.0,0
70,ES_592711_74128-1,70,1254,analizar medicamento llamado_metformina ayudar...,Este estudio analiza si un medicamento llamado...,26,0|1.0,0
156,ES_486251_63877-48,156,2817,paciente participar ensayo_clínico ayudar mejo...,Los pacientes que participan en los ensayos cl...,16,0|0.8104000091552734 9|0.06369999796152115 17|...,0
252,ES_593115_74164-9,252,4712,hijo anemia célula_falciform interés contribui...,¿Usted o su hijo tienen la anemia de células f...,26,0|1.0,0
...,...,...,...,...,...,...,...,...
60662,ES_579773_67632-8,60662,1061456,diagnosticar síndrome_takotsubo investigar cor...,¿Se le ha diagnosticado recientemente síndrome...,29,0|1.0,0
60696,ES_592738_74130-6,60696,1061991,analizar medicamento llamado_anastrozol utiliz...,Este estudio analiza si un medicamento llamado...,25,0|1.0,0
60698,ES_592759_74132-5,60698,1062015,crisis dolor complicación pulmonar anemia célu...,Crisis de dolor y complicaciones pulmonares en...,46,0|1.0,0
60740,ES_597996_74674-2,60740,1062566,sufrir enfermedad cardíaco requerir_dci preven...,¿Sufre una enfermedad cardíaca que requiere un...,24,0|1.0,0


In [19]:
df_all = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/translated_stops/df_1.parquet")
df_all = df_all[df_all.lang == "ES"]

In [20]:
from sentence_transformers import SentenceTransformer, util
import itertools

query = "¿Cuál es la mejor manera de ayudar a mi bebé a conciliar el sueño?"
docs = df_all.text.values.tolist()#list(itertools.chain(*[text.split(".") for text in df_en_raw.text.values.tolist()]))#df_tpc_es.text.values.tolist()

#Load the model
model = SentenceTransformer('SeyedAli/Multilingual-Text-Semantic-Search-Siamese-BERT-V1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)[:3]

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)


/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.8129085302352905 ¿Cuál es la mejor manera de ayudar al equipo médico?
0.8129085302352905 ¿Cuál es la mejor manera de ayudar al equipo médico?
0.8129085302352905 ¿Cuál es la mejor manera de ayudar al equipo médico?


In [21]:
query_emb = model.encode("¿Cómo puedo ayudar a dormir a mi bebé?")

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)[:10]

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)

0.7978652715682983 ¿Cómo puedo protegerme a mí misma y a mi bebé?
0.7978652715682983 ¿Cómo puedo protegerme a mí misma y a mi bebé?
0.7732670307159424 ¿Cómo puede ayudar al bebé a quedarse dormido: No todos los bebés saben cómo dormirse solos. Cuando es la hora de dormir, muchos padres mecen al bebé. Los recién nacidos y bebés pequeños se duermen mientras están amamantando. Tener una rutina para la hora de dormir es una buena idea. Pero si un bebé más grande se duerme mientras come o en brazos, puede convertirse en un patrón. Puede que el bebé empiece a esperar estar en brazos para dormirse. Cuando el bebé se despierta por poco tiempo durante un ciclo de sueño, es posible que no pueda volver a dormirse solo.
0.7732670307159424 ¿Cómo puede ayudar al bebé a quedarse dormido: No todos los bebés saben cómo dormirse solos. Cuando es la hora de dormir, muchos padres mecen al bebé. Los recién nacidos y bebés pequeños se duermen mientras están amamantando. Tener una rutina para la hora de dorm

In [27]:
from sentence_transformers import SentenceTransformer, util
import time

class Index(object):
    def __init__(self, corpus: list, doc_ids: list, model_name: str = 'SeyedAli/Multilingual-Text-Semantic-Search-Siamese-BERT-V1'):
        self.model = SentenceTransformer(model_name)
        self.corpus = corpus
        self.doc_ids = doc_ids
        
        # Preprocess the corpus: split each document into sentences
        self.sentences = []
        self.mapping = []
        for doc_id, document in enumerate(corpus):
            
            # Split the document into sentences based on "."
            substrings = document.split(".")
            substrings = [s.strip() for s in substrings if s.strip()]  #
            
            self.sentences.extend(substrings)
            self.mapping.extend([doc_id] * len(substrings))
        
        print("-- -- Encoding corpus...")
        start_time = time.time()
        self.index = self.model.encode(self.sentences)
        print("-- -- Corpus encoded in {} minutes".format((time.time()-start_time)/60))
    
    def retrieve(self, query, topk=5):
        # Encode query
        query_emb = self.model.encode(query)
        
        # Compute dot score between query and all sentence embeddings
        scores = util.dot_score(query_emb, self.index)[0].cpu().tolist()
        
        # Combine sentences & scores
        sentence_score_pairs = list(zip(self.sentences, scores, self.mapping))
        
        # Sort by decreasing score
        sentence_score_pairs = sorted(sentence_score_pairs, key=lambda x: x[1], reverse=True)[:topk]
        
        # Retrieve the topk results with original document mapping
        results = []
        for sentence, score, doc_id in sentence_score_pairs:
            results.append({
                "document_id": self.doc_ids[doc_id],
                "sentence": sentence,
                "score": score,
                "original_document": self.corpus[doc_id]
            })
        
        return results

In [33]:
index = Index(corpus=df_all.text.values.tolist(), doc_ids=df_all.doc_id.values.tolist())

-- -- Encoding corpus...
-- -- Corpus encoded in 27.29057859579722 minutes


In [34]:
index.sentences[0:10]

['Información sobre el seguro médico: Wisconsin Medicaid (solo servicios de Badgercare y planificación familiar) aceptado',
 'El tratamiento no solo se ocupa de las drogas',
 'Le ayuda a tomar el control de su vida para que no tenga que depender de las drogas',
 'Aprenderá buenos motivos para dejar las drogas',
 'Mantenerse alejado de las drogas es un proceso de por vida que implica compromiso y esfuerzo',
 'Los factores de riesgo para la enfermedad de Addison de tipo autoinmunitario incluyen otras enfermedades autoinmunitarias:\\n- Hinchazón (inflamación) de la glándula tiroides que con frecuencia lleva a una reducción en la función tiroidea (tiroiditis crónica)\\n- La glándula tiroides produce un exceso de hormona tiroidea (hipertiroidismo, enfermedad de Graves)\\n- Erupción que produce picazón con ronchas y ampollas ( dermatitis herpetiforme)\\n- Las glándulas paratiroides del cuello no producen suficiente hormona paratiroidea (hipoparatiroidismo)\\n- La hipófisis no produce cantida

In [38]:
import pickle; pickle.dump(index, open('index_store_es_all.pkl', 'wb'))

In [37]:
query = "How can I help my baby to sleep?"
results = index.retrieve(query, topk=10)

for result in results:
    print(f"Document ID: {result['document_id']}")
    print(f"Sentence: {result['sentence']}")
    print(f"Score: {result['score']}")
    print(f"Original Document: {result['original_document']}")
    print("\n")

Document ID: ES_166253_1505-3
Sentence: Entre los libros sobre estrategias CIO más populares se encuentran Solve Your Child’s Sleep Problems, de Ferber, Healthy Sleep Habits, Happy Child, de Weissbluth, y The Sleepeasy Solution, de Waldburger y Spivack
Score: 0.6411253809928894
Original Document: Hay dos métodos básicos para entrenar el sueño. El primero consiste en dejar que el niño “llore” (CIO, por sus siglas en inglés). La idea que subyace al CIO es que los padres a veces se interponen en el camino para que los niños aprendan a dormirse. Con el CIO, el bebé aprenderá a dormirse solo después de algunas noches difíciles. Los padres pueden dejar que su bebé llore y vigilarlo en periodos de tiempo que se alargan gradualmente. Entre los libros sobre estrategias CIO más populares se encuentran Solve Your Child’s Sleep Problems, de Ferber, Healthy Sleep Habits, Happy Child, de Weissbluth, y The Sleepeasy Solution, de Waldburger y Spivack.


Document ID: ES_476546_63177-5
Sentence: Sleep-r

In [14]:
import pathlib
import pandas as pd
from scipy import sparse
import numpy as np

path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/multi_blade_filtered/df_1.parquet")

path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/MULTI_BLADE_FILTERED/poly_rosie_v2_1_20")

def get_doc_top_tpcs(doc_distr, topn=2):
    sorted_tpc_indices = np.argsort(doc_distr)[::-1]
    top = sorted_tpc_indices[:topn].tolist()
    top_weight = [(k, doc_distr[k]) for k in top]
    return top_weight

def get_doc_main_topc(doc_distr):
    sorted_tpc_indices = np.argsort(doc_distr)[::-1]
    top = sorted_tpc_indices[:1][0]
    return top

In [6]:
raw = pd.read_parquet(path_source)
raw

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang
0,0,implant_port information technology easy child...,puerto_implantado facilitar hijo recibir trata...,EN_657272_107210-27,Implanted ports make it easier for you or your...,Los puertos implantados facilitan que usted o ...,EN
1,1,life_expectancy birth reach record high year t...,esperanza_vida nacer alcanzar_máximo precedent...,EN_403516_60988-2,Life expectancy at birth reached a record high...,La esperanza de vida al nacer alcanzó un máxim...,EN
2,2,shazam_hussain arterial common location caroti...,shazam_hussain arterial común provenir arteria...,EN_736110_112440-10,"Shazam Hussain: Usually, on the arterial side....","Shazam Hussain: Por lo general, en el lado art...",EN
3,3,rifaximin cause effect doctor unusual_problem ...,rifaximina causar secundario_llamir médico pro...,EN_1279033_283895-10,Rifaximin may cause other side effects. Call y...,La rifaximina puede causar otros efectos secun...,EN
4,4,point healing_process need dressing provider l...,curación necesitar apósito proveedor dejar her...,EN_1241536_280479-12,"At some point during the healing process, you ...",En algún momento durante el proceso de curació...,EN
...,...,...,...,...,...,...,...
1062898,1062898,complicación enfermedad grave afectación compl...,complication disease severe ocular complicatio...,ES_833462_8011-30,Las complicaciones de esta enfermedad pueden s...,"Complications of this disease can be severe, w...",ES
1062899,1062899,información prueba servicio embarazo centro sa...,information pregnancy test service health cent...,ES_651664_92975-2,Información sobre pruebas y servicios para el ...,Information about pregnancy tests and services...,ES
1062900,1062900,quedar beneficio secundario importante,information technology clear benefit effect im...,ES_189923_37958-96,2. ¿Le queda claro cuáles beneficios y efectos...,2. Is it clear to you what benefits and side e...,ES
1062901,1062901,limitación serie escaso número paciente anális...,limitation study information technology series...,ES_1043194_9292-51,Existen limitaciones en nuestro estudio: es un...,There are limitations in our study: it is a se...,ES


In [15]:
thetas = sparse.load_npz(path_model / "mallet_output" / "thetas_EN.npz")
raw["thetas"] = list(thetas.toarray())
raw.loc[:, "top_k"] = raw["thetas"].apply(get_doc_top_tpcs)
raw.loc[:, "main_topic"] = raw["thetas"].apply(get_doc_main_topc)

In [18]:
# get topic keys in Engish
with open(path_model / "mallet_output" / "keys_EN.txt", 'r') as file:
    lines = file.readlines()
topic_keys = [line.strip() for line in lines]
topic_keys

['doctor clinic mayo condition search minnesota rochester medical focus disease transplant surname care florida result center treat health location',
 'symptom pain technology medication medicine doctor treat severe feel infection treatment child day people sign skin include fever mild',
 'patient treatment study therapy clinical drug receive organization treat risk month trial follow evidence health rate effective day improve',
 'cancer tumor treatment breast cell radiation technology surgery therapy screening risk tissue thyroid stage treat gland prostate remove hormone',
 'disorder syndrome gene brain cell condition protein genetic affect mutation acid disease technology people function change development seizure develop',
 'baby woman pregnancy birth technology pregnant health infant risk organization week mother bear sex sexual month period delivery newborn',
 'infection hiv person report virus tuberculosis disease laboratory outbreak health testing infect cdc illness result posit

In [20]:
df_en = raw[raw['doc_id'].str.startswith("EN")].copy()
df_es = raw[raw['doc_id'].str.startswith("ES")].copy()

In [22]:
df_es[df_es.main_topic == 5]

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang,thetas,top_k,main_topic
18,18,contaminación_atmosférico causar cambio_climát...,air_pollution cause climate_change link death ...,ES_865177_116539-11,Esa misma contaminación atmosférica que causa ...,The same air pollution caused by climate chang...,ES,"[0.0, 0.0357142873108387, 0.0, 0.0, 0.0, 0.857...","[(5, 0.8571428656578064), (19, 0.0357142873108...",5
28,28,hijo control anote_fecha hora visita,child control appointment write date time purp...,ES_709235_98634-42,"Si su hijo tiene una cita de control, anote la...","If your child has a control appointment, write...",ES,"[0.0, 0.0, 0.0, 0.19230769574642181, 0.0, 0.65...","[(5, 0.6538461446762085), (3, 0.19230769574642...",5
31,31,suceder prueba,happen test procedure,ES_699215_98273-76,Qué sucedería si no se hace la prueba o el pro...,What would happen if the test or procedure is ...,ES,"[0.0, 0.0, 0.012658228166401386, 0.0, 0.0, 0.4...","[(5, 0.4810126721858978), (18, 0.4050633013248...",5
32,32,analizar_tecnología medir actividad cardíacar_...,study new technology measure fetal_cardiac act...,ES_591388_73930-2,Este estudio analiza una nueva tecnología para...,This study analyzes a new technology to measur...,ES,"[0.0, 0.23529411852359772, 0.0, 0.029411764815...","[(5, 0.3529411852359772), (1, 0.23529411852359...",5
59,59,regular comercialización alimento_bebida salud...,regulate marketing_unhealthy food_beverage pub...,ES_862069_116278-41,Regular la comercialización de alimentos y beb...,Regulate the marketing of unhealthy foods and ...,ES,"[0.07999999821186066, 0.0, 0.0, 0.0, 0.0, 0.80...","[(5, 0.800000011920929), (0, 0.079999998211860...",5
...,...,...,...,...,...,...,...,...,...,...
1062722,1062722,tratamiento convulsión diagnosticar hijo epile...,convulsion treatment child diagnose epilepsy d...,ES_336586_3064-40,Tratamiento para las convulsiones: Si diagnost...,Convulsions Treatment: If your child is diagno...,ES,"[0.0, 0.2857142984867096, 0.0, 0.1785714328289...","[(5, 0.3928571343421936), (1, 0.28571429848670...",5
1062762,1062762,exhale_empuje peso contener respiración_inhale...,exhale_push weight hold_breath time inhale lit...,ES_233066_39166-20,Exhale cuando empuje contra el peso. No conten...,Exhale when pushing against the weight. Do not...,ES,"[0.0, 0.24637681245803833, 0.0, 0.0, 0.0144927...","[(5, 0.49275362491607666), (1, 0.2463768124580...",5
1062766,1062766,persona tímido volver él_sensible sentimiento ...,shy people sensitive people feeling emotion th...,ES_364146_44036-10,Las personas tímidas también pueden volverse m...,Shy people can also become more sensitive to o...,ES,"[0.014925372786819935, 0.014925372786819935, 0...","[(5, 0.43283581733703613), (15, 0.268656730651...",5
1062789,1062789,cuidados vida tema,end life care information topic,ES_283635_40554-44,Cuidados al final de la vida: Para obtener más...,"End-of-life care: For more information, see th...",ES,"[0.0, 0.0, 0.0, 0.0, 0.008620689623057842, 0.5...","[(5, 0.5086206793785095), (17, 0.2327586263418...",5


In [23]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
import itertools

tokenizer = T5Tokenizer.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
claim_extractor = T5ForConditionalGeneration.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
classifier = pipeline(
        "text-classification", 
        model="Nithiwat/bert-base_claimbuster"
)

#lens = [len(c) for c in claims]
#texts = [[text[i]]*lens[i] for i in range(len(lens))]
#scores = [classifier(c)[0]['label'] for c in claims] 

def generate_claims(text):
    tok_input = tokenizer.batch_encode_plus([text], return_tensors="pt", padding=True)
    claims = claim_extractor.generate(**tok_input)
    claims = tokenizer.batch_decode(claims, skip_special_tokens=True)[0].split(".")

    scores = [classifier(c)[0]['label'] for c in claims] 

    return claims, scores

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/lbartolome/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [37]:
df_tpc = df_en[df_en.main_topic==19]
df_tpc = df_tpc[df_tpc['text'].str.contains('baby', case=False, na=False)]
df_tpc

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang,thetas,top_k,main_topic
411,411,birth_defect birth_defect common costly critical condition affect baby_bear united_states read learn birth_defect woman improve chance baby_bear birth_defect,defecto defecto_congénito condición común costoso crítica afectar bebé nacido unidos lea aprender defecto_congénito mujer mejorar bebé nacido defecto_congénito,EN_992239_17658-0,"What are Birth Defects: Birth defects are common, costly, and critical conditions that affect 1 in every 33 babies born in the United States each year.1 Read more about what we have learned about birth defects and how women can improve their chances of having a baby born without a birth defect.","¿Cuáles son los defectos congénitos?Los defectos congénitos son condiciones comunes, costosas y críticas que afectan a 1 de cada 33 bebés nacidos en los Estados Unidos cada año.1 Lea más sobre lo que hemos aprendido acerca de los defectos congénitos y cómo las mujeres pueden mejorar sus posibilidades de tener un bebé nacido sin un defecto congénito.",EN,"[0.0, 0.239130437374115, 0.0, 0.0, 0.0, 0.021739130839705467, 0.021739130839705467, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021739130839705467, 0.0, 0.021739130839705467, 0.0, 0.0, 0.0, 0.6739130616188049]","[(19, 0.6739130616188049), (1, 0.239130437374115)]",19
698,698,aliza fussy baby eat conk wake eat bite fall_asleep high chair,alizar bebé_quisquilloso comer contagiar levantar comer_bocado quedar_dormido silla alto,EN_611321_99695-3,"Aliza was a fussy baby, until she ate. Then she’d conk out almost immediately. She would wake up early, and after eating a bite or two, she’d fall asleep in her high chair.","Aliza era un bebé quisquilloso, hasta que comió. Entonces ella se contagiaba casi inmediatamente. Se levantaba temprano, y después de comer un bocado o dos, se quedaba dormida en su silla alta.",EN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020408162847161293, 0.0, 0.0, 0.0, 0.0, 0.0, 0.020408162847161293, 0.22448979318141937, 0.0, 0.040816325694322586, 0.020408162847161293, 0.020408162847161293, 0.6530612111091614]","[(19, 0.6530612111091614), (14, 0.22448979318141937)]",19
790,790,miss round report nurse care baby day available bedside round caregiver speaker_phone parent hear update ask question real_time ask nurse share question doctor ask day,olvidar ronda informe enfermera cuidar bebé disponible cama ronda cuidador altavoz padre escuchar actualización pregunta real pedir él enfermera compartir pregunta médico pídale llamar,EN_1420818_306928-4,"If you miss rounds, you can get a report from the nurse caring for your baby later in the day. If you are not available to be at bedside during rounds, the caregiver can use a speaker phone so that parents can hear the update and ask questions in real time. Or, you can ask your nurse to share your questions with your doctor and ask them to call you later that day.","Si se olvida de las rondas, puede obtener un informe de la enfermera que cuida a su bebé más tarde en el día. Si no está disponible para estar al lado de la cama durante las rondas, el cuidador puede usar un altavoz para que los padres puedan escuchar la actualización y hacer preguntas en tiempo real. O puede pedirle a su enfermera que comparta sus preguntas con su médico y pídale que le llame más tarde ese mismo día.",EN,"[0.0, 0.016393441706895828, 0.016393441706895828, 0.016393441706895828, 0.0, 0.0, 0.016393441706895828, 0.0, 0.032786883413791656, 0.08196721225976944, 0.0, 0.04918032884597778, 0.0, 0.0, 0.016393441706895828, 0.0, 0.0, 0.1147540956735611, 0.0, 0.6393442749977112]","[(19, 0.6393442749977112), (17, 0.1147540956735611)]",19
1069,1069,household safety prevent injury crib baby spend lot time crib nap day sleep_night information technology important sure information technology safe environment place baby sleep help prevent sudden infant death syndrome_sids,seguridad hogar prevención lesión cuna bebé pasar cuna dormir_siesta dormir_noche importante asegurar

In [38]:
all_ids = []
all_texts = []
all_claims = []
all_scores = []

for id_, el in df_tpc.iterrows():
    claims, scores = generate_claims(el.text)
    texts = [el.text]*len(claims)
    ids = [el.doc_id]*len(claims)

    all_ids.append(ids)
    all_texts.append(texts)
    all_claims.append(claims)
    all_scores.append(scores)


results = pd.DataFrame(
    {
        "ids": list(itertools.chain(*all_ids)),
        "text" : list(itertools.chain(*all_texts)),
        "claims" : list(itertools.chain(*all_claims)),
        "score" : list(itertools.chain(*all_scores))
    }
)

In [39]:
filtered = results[results.score == "LABEL_1"]
filtered

,ids,text,claims,score
0,EN_992239_17658-0,"What are Birth Defects: Birth defects are common, costly, and critical conditions that affect 1 in every 33 babies born in the United States each year.1 Read more about what we have learned about birth defects and how women can improve their chances of having a baby born without a birth defect.","Birth defects are common, costly, and critical conditions",LABEL_1
1,EN_992239_17658-0,"What are Birth Defects: Birth defects are common, costly, and critical conditions that affect 1 in every 33 babies born in the United States each year.1 Read more about what we have learned about birth defects and how women can improve their chances of having a baby born without a birth defect.",Birth defects affect 1 in every 33 babies born in the United States each year,LABEL_1
6,EN_611321_99695-3,"Aliza was a fussy baby, until she ate. Then she’d conk out almost immediately. She would wake up early, and after eating a bite or two, she’d fall asleep in her high chair.",She’d conk out almost immediately,LABEL_1
20,EN_903854_138864-2,"Just had a baby or adopted Are under 30 Have/offered job-based insurance Are self-employed Are unemployed Qualify for Medicare Taxes Healthcare & taxes Form 1095-A 'Reconcile' tax credit Browse all topics Featured Find out if you qualify for a Special Enrollment Period 2023 Open Enrollment is over, but you may still be able to enroll in 2023 health insurance through a Special Enrollment Period. Get Answers Search Back to glossary Care coordination The organization of your treatment across several health care providers. Medical homes and Accountable Care Organizations are two common ways to coordinate care.",Just had a baby or adopted Are under 30 Have/offered job-based insurance Are self-employed Are unemployed Qualify for Medicare Taxes Healthcare & taxes Form 1095-A 'Reconcile' tax credit Browse all topics Featured Find out if you qualify for a Special Enrollment Period 2023 Open Enrollment is over,LABEL_1
21,EN_903854_138864-2,"Just had a baby or adopted Are under 30 Have/offered job-based insurance Are self-employed Are unemployed Qualify for Medicare Taxes Healthcare & taxes Form 1095-A 'Reconcile' tax credit Browse all topics Featured Find out if you qualify for a Special Enrollment Period 2023 Open Enrollment is over, but you may still be able to enroll in 2023 health insurance through a Special Enrollment Period. Get Answers Search Back to glossary Care coordination The organization of your treatment across several health care providers. Medical homes and Accountable Care Organizations are two common ways to coordinate care.",You may still be able to enroll in 2023 health insurance through a Special Enrollment Period,LABEL_1
...,...,...,...,...
12023,EN_1397715_301977-1,"A new routine: If possible, create a pumping routine based on when your baby would normally breastfeed, especially when first returning to work. But you, your baby, and your milk production will adjust to a new routine if you are able to pump often enough. Many mothers do find pumping sessions go more quickly when they are able to pump at about the same time each day.",Many mothers do find pumping sessions go more quickly when they are able to pump at about the same time each day,LABEL_1
12025,EN_1207649_277801-3,"Like many surgical procedures, cesarean sections require anesthesia. Usually, the mother is given an epidural or a spinal block. Both of these will numb the lower body, but the mother will remain awake. If the baby has to be delivered quickly, as in an emergency, the mother may be given a general anesthetic, which will make her fall asleep. During the surgery, an incision is made in the lower abdomen followed by an incision made in the uterus. There is no pain associated with either of these incisions because of the anesthesia.",Cesarean sections require anesthesia,LABEL_1
12033,EN_1207649_277801-3,"Like many surgical procedures, cesarean sections require anesthesia. Usually, the mother is 

In [ ]:
class GenerateQuestion(dspy.Signature):
    fact = dspy.InputField()
    context = dspy.InputField()
    question = dspy.OutputField(desc="it asks the fact", prefix="Question:")


class QAGeneratorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_question = dspy.Predict("fact,context->question")

    def forward(self, fact,context):
        question = self.generate_question(fact=fact, context=context).question
        return dspy.Prediction(question=question)

qa_gen = QAGeneratorModule()

class TranslatorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.translate = dspy.Predict("english->spanish")

    def forward(self, english):
        spanish = self.translate(english=english).spanish
        return spanish
tr = TranslatorModule()
tr("What is the best way to help your baby fall asleep?")

In [ ]:
questions = [qa_gen(q,con).question for q,con in zip(filtered.claims.values.tolist(),filtered.text.values.tolist())]
filtered["question"] = questions

filtered["len"] = filtered["question"].apply(lambda x: len(x))
filtered_2 = filtered[
    (filtered["len"] <= 300) &
    (filtered["question"].str.endswith("?"))
].drop_duplicates(subset=['question'], keep='first', inplace=False)
filtered_2.to_excel("test_tpc5_multi.xlsx")